In [10]:
import math
from typing import Any

import torch

from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch import functional as F

from torch.nn import Module
from scipy.stats import pareto 
from torch.distributions.pareto import Pareto
from torch.nn import init
seed=0
#torch.cuda.manual_seed_all(seed)

#emptying any left over file from previous run
with torch.cuda.device('cuda:0'):
    torch.cuda.empty_cache()

import datetime
import os

import torch
import torchvision
#torch.cuda.is_available()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
#from torch.autograd.functional import jacobian
#import torch.autograd.functional
import time
import copy
import matplotlib.pyplot as plt
import pandas as pd
    

In [ ]:
def add_gaussian_noise(inputs,strength):
    #add guassian noise to input image

    #print(inputs.size())
    noise = torch.randn_like(inputs)*strength*torch.max(inputs)
    result=inputs + noise
    return result


def add_powerlaw_noise(input_size,batch_size,strength,images):
    ##add power law noise(not use for not )

    ##input size here is the side of the image 28 
    m = Pareto(torch.tensor([1.0]), torch.tensor([1.0]))
    res=m.sample(images.size())

    temp=res/np.sqrt(res.var())/N/batch_size*strength
    noise=temp-temp.mean()
    #if print_bool:
     #   plt.title(powerlaw_noise print)
      #  plt.hist(noise.flatten(),density=True, bins='auto', histtype='stepfilled')
        
      #  plt.show()
    #print(noise[:,:,:,:,0].size())
    return noise[:,:,0]+images

def validation(test_loader):
    #checking the accuracy of prediction on the test dataset
    with torch.no_grad():

     #valid_loss = criterion(outputs, batch_features.view(batch_size,-1).to(device))
        loss=0
        for batch_features, _ in test_loader:
            #input=add_noise(batch_features,).view(100, -1).to(device)
            input_image=add_gaussian_noise(batch_features,noise_strength).view(batch_size, -1).to(device)
            #batch_feature = batch_features.view(batch_size, -1)
            #input_image =add_powerlaw_noise(input_size,batch_size,noise_strength,batch_feature).view(batch_size, -1).to(device)
            batch_feature = batch_features.view(batch_size, -1).to(device)
            outputs = model(input_image)
            valid_loss = criterion(outputs, input_image.to(device)) 
            #accuracy+=kl_div(outputs,batch_features)/batch_size
            loss += valid_loss.item()
    return loss/len(test_loader)
            
def kl_div(output_image,input_image):
    #one way to check how accurate
    accuracy=0
    kl_loss = nn.KLDivLoss(reduction="batchmean")
    for i in range(len(input_image)):
        input_spectrum=torch.histogram(input_image[i].cpu(), bins=256, density=True)
        output_spectrum=torch.histogram(output_image[i].cpu(), bins=256, density=True)
        accuracy+=kl_loss(input_spectrum[i],output_spectrum[i])
    return accuracy/len(input_image)

def iteration(model,initial_inputs,no_of_repetition,epoch):
    y_pred=initial_inputs.to(device)
    for i in range(no_of_repetition):
        x_train=y_pred
        #if i==0:
           # name="progress asymptotic iteration:"+str(i)+" "
           # visualisation(x_train[0].cpu().detach(),epoch,name,False)
        #if i%5==1:
            #name="progress asymptotic iteration:"+str(i)+" "
            #visualisation(x_train[0].cpu().detach(),epoch,name,False)
        y_pred=model.internals(x_train)
    return x_train,y_pred,model 

def asymptotic_jacobian(model,initial_input,no_of_images,no_of_repetition):
    ave_jac=0
    overall_distribution=np.asarray([])

    jacobian=[]
    sorteds=np.asarray([])
    for j in range(no_of_repetition):
        #print("here")
        x_train=initial_input.to(device)
        y_pred=x_train
        mean_jacobian=0
        for i in range(no_of_images):
            x=x_train[i]
            res=torch.autograd.functional.jacobian(model.internals,x)
            distribution=np.asarray([])
            jacobian.append(res)
            sorted=np.asarray([])
            #if j==no_of_repetition-1:
             #   distribution,sorted=spectral_distribution(res.cpu())
              #  overall_distribution=np.concatenate((overall_distribution,distribution),axis=0)
              #  spectral=spectral_radius(sorted)
              #  sorteds=np.append(sorteds,spectral)

            norm=torch.norm(res).cpu()
            mean_jacobian+=1/np.sqrt(output_size)*norm
        mean_jacobian=mean_jacobian/no_of_images
        ave_jac+=mean_jacobian
    average_jacobian=ave_jac/no_of_repetition
    return average_jacobian,overall_distribution,jacobian,sorteds

def spectral_radius(jacobian,no_of_repetition,no_of_images):
    #result=[]
   
    #output=initial_input.to(device)
   # product=torch.eye(n=input_size,m=output_size).to(device)
    
    spectral=0

    
        
    #res=torch.autograd.functional.jacobian(model.internals,initial_input)
    #product=torch.matmul(product,res)
    #output=model(initial_input)
   # initial_input=output
    
    s,v=torch.linalg.eig(jacobian)
 #   #print(s)
    abs=torch.abs(s)
    
    spectral=torch.max(abs).item()
    return spectral

def poincare_plot(model,initial_input,dimension_vector,no_of_repetition,colour,epoch):
    '''
    TODO: add absolute for xt and xtminus
    Calculate the poincare plot of the model

    Parameters
    ----------
        model: torch.nn.Module
            NN model used (autoencoder)
        initial_input:  
            The initial input image
        dimension_vector: torch.Tensor
            The dimension vector for the poincare plot
        no_of_repetition: int
            number of repetitions for images
        colour: Numpy array
            Array to specify the colour of each point
        epoch: int
            Number of epochs

    Returns
    -------
        xt: list
            The x coordinate of the poincare plot
        xtminus: list
            The y coordinate of the poincare plot

    '''
    xt=[]
    xtminus=[]
    
    output=initial_input
    
    for i in range(no_of_repetition):
        
        initial_input=output
        output=model(initial_input)
        
        xt.append(np.abs((1/output_size*torch.dot(output,dimension_vector).item())))
        xtminus.append(np.abs(1/output_size*torch.dot(initial_input,dimension_vector).item()))
    
    return xt,xtminus
  
    


def asymptotic_distance(xinfinity_unperturbed,xinfinity_perturbed,perturbation):
    result=[]
    for i in range(len(xinfinity_unperturbed)):
    
        sum=0
        
        for j in range(len(xinfinity_unperturbed[i])):
            
            temp=np.linalg.norm(xinfinity_unperturbed[i][j]-xinfinity_perturbed[i][j])
            sum+=temp
        
        result.append(1/output_size*1/len(xinfinity_unperturbed[i])*sum) 
    
    return result


def calculation(model,no_of_images,no_of_repetition,ave_jacobian,spectral_radiuses,image,epoch):
    spectral=0
    
    model_clone=AE(input_size,input_size,hidden_state_size,no_of_layer).to(device)
    model_clone.load_state_dict(copy.deepcopy(model.state_dict()))
    distributions=np.asarray([])
    x_train,y_pred,model_clone=iteration(model_clone,image,asymptotic_repetition,epoch)   
    #ave_jac,distributions,jacobian,sorteds= asymptotic_jacobian(model_clone,x_train,no_of_images,no_of_repetition)
    
    #np.savetxt("jacobian epoch"+str(epoch)+".txt",sorteds)
    #print("distribution size")
    #print(distributions.size)
    #print("sorted size")
    #print(sorteds.size)
    #average_jacobian.append(ave_jac)
    #x = [ele.real for ele in distributions]
    ## extract imaginary part
    #y = [ele.imag for ele in distributions]
    #plt.title("real and imaginary part of eigenvalue")
    #plt.scatter(x, y)
    #plt.ylabel('Imaginary')
    #plt.xlabel('Real')
    #plt.xscale("log")
    #plt.yscale("log")
    #plt.xlim(-1,1)
    #plt.ylim(-1,1)
    #plt.savefig("epoch:"+str(epoch+1)+"number of iteration:"+str(no_of_repetition)+"eigenvalue scatter plot.jpg",bbox_inches = 'tight')
    #plt.show()
    #plt.title("modulus of eigenvalue in log log plot ")
    #plt.hist(sorteds, density=True, bins='auto', histtype='stepfilled')
    #plt.yscale("log")
    #plt.xscale("log")

    #plt.savefig("epoch:"+str(epoch+1)+"number of iteration:"+str(no_of_repetition)+"eigenvalue distribution.jpg",bbox_inches = 'tight')
    #plt.show()

   # for i in range(no_of_images):
    
        #spectral+=spectral_radius(sorteds,no_of_repetition,no_of_images)
    #spectral=sorteds.mean()
    
    #spectral_radiuses.append(spectral)
    #print(spectral/no_of_images)
    
    return x_train,y_pred,ave_jacobian,spectral_radiuses,model_clone


def asymptotic_iteration(model_new,initial_inputs,perturbed_inputs,no_of_repetition,no_of_image,cutoff):
    
    y_pred_unperturbed=initial_inputs.to(device)
    y_pred_perturbed=perturbed_inputs.to(device)
    #cutoff=[]
   
    #for j in range(no_of_image):
     
    #cutoff.append(torch.norm(y_pred_unperturbed[j]-y_pred_perturbed[j]))
        #print(cutoff[j])
    distance=[]
    hidden_layer=[]
    hiddens=[]
    chaos=0
 
    for i in range(no_of_repetition):
        x_train_unperturbed=y_pred_unperturbed
        y_pred_unperturbed=model_new.internals(x_train_unperturbed)
        
       
        x_train_perturbed=y_pred_perturbed
        y_pred_perturbed=model_new.internals(x_train_perturbed)
        
       # print(y_pred_perturbed,y_pred_unperturbed)
        
        hidden = model.hiddens()
        #print(hidden)
        #print(torch.sum(hidden,dim=1).cpu().detach().numpy()/1000)
        hidden_layer.append(torch.sum(hidden,dim=1)[0].cpu().detach().numpy()/10)
        hiddens.append(torch.sum(hidden,dim=1).cpu().detach().numpy()/10)
        
        #visualisation(y_pred.view(batch_size,-1)[0].cpu().detach(),epoch)


        #diff=0
        diff=torch.zeros(1,device=device)
        temp=0
        #print(len(cutoff))
        for j in range(no_of_image):
            result=torch.norm(y_pred_unperturbed[j]-y_pred_perturbed[j])
            #print(np.isnan(result.cpu().detach().numpy()))
            #print(result)
            if np.isinf(result.cpu().detach().numpy()) or np.isnan(result.cpu().detach().numpy())  :
                result=torch.Tensor([1*10**38]).to(device)
                print("infinity")
            temp=result
            diff+=result
            
            #print(j)
            if result>cutoff[j] and i==no_of_repetition-1:
                chaos+=1
            if np.isinf(diff.cpu().detach().numpy()) or np.isnan(diff.cpu().detach().numpy()):
                diff=torch.Tensor([1*10**38]).to(device)
                print("infinity")
            #print("asymptotic calculation")
            #print(result.cpu())
            #print(diff.cpu())
            
        if diff.cpu().detach().numpy()<2**-52:
            diff=2**-52
            distance.append(diff)
            continue
        distance.append(diff.cpu().detach().numpy()/no_of_image)

    return distance,hidden_layer,chaos/no_of_image

def visualisation(xfinals,epoch,name,bool):
    plt.imshow(xfinals.reshape(N,N), cmap="gray")
    if bool:
        plt.savefig(str(name)+" epoch:"+str(epoch+1)+".jpg")
    plt.show()
    print(str(name)+" epoch:"+str(epoch+1)+".jpg")
    
def divergence(values):
    result=np.abs(values[-1]-values[-2])
    if result>np.abs(values[2]-values[1]):
        return 1
    return 0

def spectral_radius(sorted):
    if len(sorted)==0:
        return 0
    return sorted[0]

def spectral_distribution(input_matrix):
    result=[]
    count=0
    s,v=torch.linalg.eig(input_matrix)
    #return s
    for i in range(len(s)):
        if np.abs(s[i].cpu())<2**-52:
            count+=1
        
    sorted, indices=torch.sort(torch.abs(s),dim=-1,descending=True)
    #for index in indices.cpu():
     #   if index<=len(s)-count:
     #       result.append(s[index])
    #print(sorted)
    return s,sorted[:-count]



In [28]:
## Brent's cycle detection algorithm

##TODO distunguish between period cycle, chaos and convergence
##TODO automated tolerance (how much they converge lapraouve exponent)


class BrentCycleDetection():
    '''
    Class for detecting cycles in a sequence of values using Brent's method.

    Parameters
    ----------
    tolerance : float
        The tolerance for closeness between two numbers. Default is 1e-2.

    Attributes
    ----------
    max_closeness : float
        Max closeness before defining as converging
    tolerance : float
        The tolerance for closeness between two numbers.
    lam : int
        The length of the cycle.
    
    mu : int
        The index of the first element of the cycle.
    
    result: str
        The result of the cycle detection. Either "Convergent", "Periodic", or "Chaotic".
    Methods
    -------
    set_lambda_mu(lam, mu)
        Sets the length of the cycle and the index of the first element of the cycle.
    
    get_lambda_mu()
        Returns the length of the cycle and the index of the first element of the cycle.

    set_result(result)
        Sets the result of the cycle detection.

    get_result()
        Returns the result of the cycle detection.

    closeness(x, y)
        Returns True if the two numbers (x,y) are within a certain tolerance of each other.
    
    detect_cycle(arr)
        Uses Brent's method to find a cycle of values within an array. Returns the length of the cycle and the index of the first element of the cycle.
    '''
    def __init__(self, max_closeness=1e-7):
        self.max_closeness = max_closeness
        self.tolerance = max_closeness
        self.lam = None
        self.mu = None
        self.result = "Chaotic"

    def get_max_closeness(self):
        '''
        Return max closeness before deciding convergence

        Returns
        -------
        float
        '''
        return self.max_closeness


    def set_tolerance(self, tolerance):
        '''
        Sets the tolerance for closeness between two numbers.

        Parameters
        ----------
        tolerance : float
            The tolerance for closeness between two numbers.
        '''
        self.tolerance = tolerance

    def get_tolerance(self):
        '''
        Returns the tolerance for closeness between two numbers.

        Returns
        -------
        float
        '''
        return self.tolerance
    
    def set_lambda_mu(self, lam, mu):
        '''
        Sets the length of the cycle and the index of the first element of the cycle.

        Parameters
        ----------
        lam : int
            The length of the cycle.
        mu : int
            The index of the first element of the cycle.
        '''
        self.lam = lam
        self.mu = mu

    def get_lambda_mu(self):
        '''
        Returns the length of the cycle and the index of the first element of the cycle.

        Returns
        -------
        lam : int
            The length of the cycle.
        mu : int
            The index of the first element of the cycle.
        '''
        return self.lam, self.mu
    
    def set_result(self, result):
        '''
        Sets the result of the cycle detection.

        Parameters
        ----------
        result : str
            The result of the cycle detection. Either "Convergent", "Periodic", or "Chaotic".
        '''
        self.result = result
    
    def get_result(self):
        '''
        Returns the result of the cycle detection.

        Returns
        -------
        str
        '''
        return self.result
    
    def closeness(self, x, y):
        '''
        Returns True if the two numbers (x,y) are within a certain tolerance of each other.

        Parameters
        ----------
        x : float
        y : float

        Returns
        -------
        bool
        '''
        return abs(x - y) < self.get_tolerance()
    
    def compute_tolerance(self, arr):
        '''
        Computes the tolerance for closeness between two numbers.

        Parameters
        ----------
        arr : list

        '''
        
        range_arr = max(arr) - min(arr)
        if range_arr == 0:
            self.set_tolerance(0)
        else:
            self.set_tolerance(1*10**(math.floor(math.log(range_arr,10) - 2)))
    


    def compute_cycle(self, arr, start=0):
        '''
        Uses Brent's method to find a cycle of values within an array.

        Parameters
        ----------
        arr : list

        start : int
            The starting index of the array. Default is 0.

        Returns
        -------
        lam : int
            The length of the cycle.
        mu : int
            The index of the first element of the cycle.
        '''
        # No cycle if the array is empty
        n = len(arr)
        if n == 0:
            self.set_lambda_mu(0, 0)
            return self.get_lambda_mu()
        
        # Main phase: search successive powers of two
        power = 1
        lam = 1
        tortoise_index = start % n
        hare_index = (start + 1) % n # Ensure the index wraps around
        while not self.closeness(arr[tortoise_index], arr[hare_index]):
            if power == lam:
                tortoise_index = hare_index
                power *= 2
                lam = 0
            hare_index = (hare_index + 1) % n # Ensure the index wraps around
            lam += 1

        # Find the position of the first repetition of length lambda
        mu = start % n
        tortoise = hare = arr[start % n]
        for i in range(lam):
            hare = arr[min(power, len(arr)-1)]
        while not self.closeness(tortoise, hare):
            tortoise = arr[min(mu, len(arr)-1)]
            hare = arr[min(mu + lam, len(arr)-1)]
            mu += 1
        self.set_lambda_mu(lam, mu)
        
        return self.get_lambda_mu()
    
    def detect_cycle(self, arr):

        #computing tolerance and returns a convergence if tolerance is too small
        self.compute_tolerance(arr)
        if self.get_tolerance() < self.get_max_closeness():
            self.set_lambda_mu(0,0)
            self.set_result("Convergent")
            return self.get_lambda_mu()
        
        #compute cycle
        self.compute_cycle(arr)
        lam, mu = self.get_lambda_mu()
        if int(mu) == int(len(arr)):
            self.set_result("Chaotic")
        elif lam == 1:
            self.set_result("Convergent")
        
        else:
            new_lam, new_mu = self.compute_cycle(arr, mu)
            new_new_lam,new_new_mu = self.compute_cycle(arr, new_mu)
            #print(new_new_lam,new_lam, lam)

            if new_new_lam == new_lam == lam:
                self.set_result("Periodic")
            elif new_new_lam <= new_lam <= lam:
                self.set_result("Convergent")
            elif new_new_lam == new_lam or new_lam == lam or lam == new_new_lam:
                self.set_result("Periodic")
            else:
                self.set_result("Chaotic")

            self.set_lambda_mu(lam, mu)
            
        return self.get_lambda_mu()
    
    def print_result(self):
        '''
        Prints the cycle of values within an array.
        '''
        lam, mu = self.get_lambda_mu()

        if self.get_result() == "Convergent":
            print("The sequence is convergent.")
        elif self.get_result() == "Periodic":
            print("Cycle found.")
            print("Cycle length:", lam)
            print("Cycle start index:", mu)
        else:
            print("The sequence is chaotic.")

        print("Cycle length:", lam)
        print("Cycle start index:", mu)

    

In [5]:
run_date = "2024-09-23_00-28-32"
colour=np.arange(1,100+1)

In [29]:
from itertools import cycle
from regex import B


epoch = 800
interval = 5
no_of_images = 100

for i in range(5,epoch+1,interval):
    print("Epoch: ", i)
    print("----------------------")

    cycle_detection = BrentCycleDetection()
    x_cycle = []
    y_cycle = []
    for j in range(no_of_images):
        xt = np.loadtxt(run_date +"/" + "image_"+ str(j)+ "/" + "xt_" + str(i) + ".txt")
        xtminus = np.loadtxt(run_date +"/" + "image_"+ str(j)+ "/" + "xtminus_" + str(i) + ".txt")
        yt = np.loadtxt(run_date +"/" + "image_"+ str(j)+ "/" +  "yt_" + str(i) + ".txt")
        ytminus = np.loadtxt(run_date +"/" + "image_"+ str(j)+ "/" + "ytminus_" + str(i) + ".txt")
        cycle_detection.detect_cycle(xt)
        x_cycle.append(cycle_detection.get_result())
        cycle_detection.detect_cycle(yt)
        y_cycle.append(cycle_detection.get_result())
    print("X Cycle")
    print("-------")
    print(x_cycle)
    print("Convergent count: "+ str(x_cycle.count("Convergent")))
    print("Periodic count: "+ str(x_cycle.count("Periodic")))
    print("Chaotic count: "+ str(x_cycle.count("Chaotic")))
    print("Y Cycle")
    print("-------")
    print(y_cycle)
    print("Convergent count: "+ str(y_cycle.count("Convergent")))
    print("Periodic count: "+ str(y_cycle.count("Periodic")))
    print("Chaotic count: "+ str(y_cycle.count("Chaotic")))
    print("----------------------")




Epoch:  5
----------------------
X Cycle
-------
['Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent', 'Convergent'